In [3]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 1.3MB 5.8MB/s 


In [4]:
jobName = '201216_Train_split1'
# jobName = '201216_Train_pilot'

# 단계1. 베이스 : 원핫 -> 스케일러
x, y = LibData.GetXY_s_o(jobName)

# x = x.sample(10000)
# y = y.loc[x.index]

def Logarithm(y):
    yLogF = np.log1p(y.pv_fee)
    yLogC = np.log1p(y.pv_claim)
    yLog = pd.DataFrame({'pv_fee':yLogF, 'pv_claim':yLogC}, index=y.index)
    return yLog

def Exponential(yLog):
    yF = np.expm1(yLog.pv_fee)
    yC = np.expm1(yLog.pv_claim)
    y = pd.DataFrame({'pv_fee':yF, 'pv_claim':yC}, index=yLog.index)
    return y


In [6]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 1002, min_samples_leaf = 1, max_features=30)

# 기존
print(LibML.GetScoresHoldOut(x,y,model,0.2))

# 변경
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2)

yTrainLog = Logarithm(yTrain)
predLog = LibML.GetPredBase(xTrain, yTrainLog, xTest, model)
pred = Exponential(predLog)
print(LibML.GetScoresOV(yTest, pred))



[169.22, 0.2019, 0.9112]
[167.02, 0.2014, 0.8505]


In [7]:
LibUtils.ELoop()

start empty loop....

KeyboardInterrupt: ignored